# Отчет по лабораторной работе №2. Наложение фильтров на видеоряд, склейка видео.
### Логинов Сергей
###  Наливайко Сергей
### Гафиров Абдималик

## Задание

**1** Наложить на видеоряд фильтр, с параметрами, меняющимися по времени.

Примеры: контрастность, с колебанием интенсивности; сдвиг цвета, с меняющимися цветами.



**2** Сделать склейку двух видеорядов, использовав один из фильтров изображения и один из визуальных эффектов смены кадров.

Примеры смены кадров: постепенный наезд нового кадра, фрагментация изображения, размытие и фокусировка на новом изображении, эффект перелистывания страницы

In [1]:
from datetime import timedelta
from moviepy.editor import *
from scipy.ndimage import gaussian_filter
from PIL import Image
from pathlib import Path
from moviepy.video.tools.credits import credits1
import numpy as np
import os
import cv2
import glob

# 1

In [2]:
def format_timedelta(td):
    """Служебная функция для форматирования объектов timedelta (например, 00: 00: 20.05)
    исключая микросекунды и сохраняя миллисекунды
    """
    result = str(td)
    
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

In [3]:
def frame_cut(video):
    """
    Функция режет видео на кадры и кладет их в папку
    video - путь до видеофайла
    """
    filename, _ = os.path.splitext(video)
    filename += '_cut'
    if not os.path.isdir(filename):
        os.mkdir(filename)
    
    video = VideoFileClip(video)
    # шаг, подходящий для фпс видео
    step = 1 / video.fps
    for cur_time in np.arange(0, video.duration, step):
        frame_duration_formatted = format_timedelta(timedelta(seconds=cur_time)).replace(":", "-")
        frame_filename = os.path.join(filename, f"frame{frame_duration_formatted}.jpg")
        video.save_frame(frame_filename, cur_time)

### Получим кадры из тестового видео. Результаты в папке test2_cut

In [4]:
frame_cut('test2.mp4')

In [5]:
def make_video_from_frames(path_to_frames, frame_size, fps):
    """
    Функция склеивает кадры в видео
    path_to_frames - путь до директории с кадрами
    frame_size - разрешение видео - кортеж типа (ширина, высота)
    fps - количество кадров в секунду
    """
    frameSize = frame_size
    out = cv2.VideoWriter('output_video_'+path_to_frames+'.mp4',
                          cv2.VideoWriter_fourcc(*'mp4v'), fps, frameSize)
    
    for filename in sorted(glob.glob(path_to_frames + '/*.jpg')): 
        img = cv2.imread(filename) 
        out.write(img) 
    
    out.release()
    cv2.destroyAllWindows()

### Создадим видео из кадров, полученных в результате выполнения прошлой функции. Результат - output_video_test_cut.mp4

In [6]:
make_video_from_frames('test2_cut', (1920, 1080), 60)

### Изменение контрастности

In [7]:
def change_contrast(path_to_frames, start, stop):
    """
    Функция покадрово меняет контрастность видео, результат сохраняется в папке
    video - используется для названия файла, функция работает с учетом frame_cut
    start - начало интервала для генерации случайного коэффициента контрастности
    stop - конец интервала для генерации случайного коэффициента контрастности
    """
    filename = 'test_contrast'
    if not os.path.isdir(filename):
        os.mkdir(filename)
    
    for filename in sorted(glob.glob(path_to_frames + '/*.jpg')):
        img = Image.open(filename)
        filename = filename[filename.find('frame'):]
        img_arr = np.array(img)

    # обработка изображения
        mean = np.mean(img_arr)
        std = np.std(img_arr)
        factor = np.random.uniform(start, stop)  # You can adjust the contrast by changing this value
        adjusted_img_arr = (img_arr - mean) * factor + mean

    # обратная конвертация в PIL image object
        adjusted_img = Image.fromarray(np.uint8(adjusted_img_arr))
        adjusted_img.save('test_contrast/'+str(filename))

### Изменим контрастность кадров и создадим видео на их основе. Результат - output_video_test_contrast.mp4

In [8]:
change_contrast('test2_cut', 1, 3)

In [9]:
make_video_from_frames("test_contrast", (1920, 1080), 60)

### Изменение цвета (соотношения RGB)

In [10]:
def make_diff_color(img, color): 
    """
    Функция применяет фильтр изменения цвета по ключу для каждого пикселя img: исходное изображение
    color - используемый цвет. Варианты: red, green, blue
    """
    width, height = img.size
    colored_img = Image.new('RGB', (width, height), (0, 0, 0)) 
    
    if color == 'red':
        for i in range(0, width):
            for j in range(0, height):
                r,g,b = img.getpixel((i, j))
                colored_img.putpixel((i,j), (255, g, b))
    elif color == 'green':
        for i in range(0, width):
            for j in range(0, height):
                r,g,b = img.getpixel((i, j))
                colored_img.putpixel((i,j), (r, 255, b))
    elif color == 'blue':
        for i in range(0, width):
            for j in range(0, height):
                r,g,b = img.getpixel((i, j))
                colored_img.putpixel((i,j), (r, g, 255))
    else:
        return 'error'
    return colored_img

def change_color(path_to_frames):
    """
    Функция меняет соотношение цветов в видео
    Видео делится на 3 равные (либо 2 равные + остаток) части,
    каждая часть "красится" своим цветом
    Все обработанные кадры сохраняются в папку
    """
    filename = "test_color"
    if not os.path.isdir(filename):
        os.mkdir(filename)
    
    # количество файлов в директории
    n = sum(1 for x in Path(path_to_frames).iterdir())
    # получение 1/3 и 2/3 кадров видео в виде списка
    sample_size = list(map(lambda x: int(x), [n/3, 2 * n/3]))
    # первая треть красится в красный
    for filename in sorted(glob.glob(path_to_frames + '/*.jpg'))[0:sample_size[0]]:
        img = Image.open(filename)
        filename = filename[filename.find('frame'):]
        adjusted_img = make_diff_color(img, 'red')
        adjusted_img.save("test_color/"+str(filename))
    # вторая треть красится в зеленый
    for filename in sorted(glob.glob(path_to_frames + '/*.jpg'))[sample_size[0]:sample_size[1]]:
        img = Image.open(filename)
        filename = filename[filename.find('frame'):]
        adjusted_img = make_diff_color(img, 'green')
        adjusted_img.save("test_color/"+str(filename))
    # остаток красится в синий
    for filename in sorted(glob.glob(path_to_frames + '/*.jpg'))[sample_size[1]:]:
        img = Image.open(filename)
        filename = filename[filename.find('frame'):]
        adjusted_img = make_diff_color(img, 'blue')
        adjusted_img.save("test_color/"+str(filename))
    cv2.destroyAllWindows()

### Изменим соотношение RGB у тестового видео. Результат - output_video_test_color.mp4

In [12]:
# дольше всего выполняется, имеет смысл проверять самый маленький test.mp4 вместо других
frame_cut('test.mp4')

In [ ]:
change_color('test_cut')

In [ ]:
make_video_from_frames('test_color', (1920, 1080), 60)

### Нарастающий эффект зернистости

In [19]:
def make_grain(img, intensity):
    """
    Функция добавляет эффект зернистости к изображению
    img: исходное изображение
    intensity: интенсивность зернистости
    """
    image = np.array(img)
    noise = np.random.rand(*image.shape) * intensity
    result = np.clip(image + noise * 255, 0, 255).astype(np.uint8)
    result = Image.fromarray(result)
    return result

def make_grain_frames(path_to_frames):
    """
    Функция покадрово накладывает эффект зернистости на видео
    Эффект (интенсивность) увеличивается со временем
    Создается директория с итоговыми кадрами, которые позже можно объединить в видео
    path_to_frames - путь к кадрам видео
    """
    # создадим массив коэффициентов интенсивности в зависимости от
    # количества кадров в видео
    n = sum(1 for x in Path(path_to_frames).iterdir())
    intens = [i * 10 / n for i in range(0, n)]
    i = 0
    
    filename = "test_grain"
    if not os.path.isdir(filename):
        os.mkdir(filename)
    
    for filename in sorted(glob.glob(path_to_frames + '/*.jpg')):
        img = Image.open(filename)
        grain_img = make_grain(img, intens[i])
        filename = filename[filename.find('frame'):]
        # Convert the adjusted image back to a PIL image object
        # Save the adjusted image
        grain_img.save("test_grain/"+str(filename))
        i += 1
    cv2.destroyAllWindows()

### Применим эффект зернистости, нарастающий со временем, к тестовому видео. Результат - output_video_test_grain.mp4

In [20]:
make_grain_frames('test2_cut')

In [ ]:
make_video_from_frames('test_grain', (1920, 1080), 60)

# 2

In [21]:
def make_concat(video1, video2):
    """
    Функция объединяет video1 и video2
    """
    video1 = cv2.VideoCapture(video1)
    video2 = cv2.VideoCapture(video2)

    # получаем фпс и разрешение на основе 1 видео
    fps = video1.get(cv2.CAP_PROP_FPS)
    width = int(video1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video1.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # объект VideoWriter для сохранения результата
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    final_video = cv2.VideoWriter('concat.mp4', fourcc, fps, (width, height))

    # само объединение
    while True:
        ret1, frame1 = video1.read()
        if not ret1:
            break
        final_video.write(frame1)

    while True:
        ret2, frame2 = video2.read()
        if not ret2:
            break
        final_video.write(frame2)

    final_video.release()
    cv2.destroyAllWindows()

### Склеим два видео

In [23]:
make_concat('test.mp4', 'test2.mp4')

### Объединение с затуханием (до черного кадра). Затухает первое видео. Не хватило времени на реализацию, тут использованы готовые функции

In [122]:
def make_black_fade_out_concat(video1, video2, time):
    """
    Функция объединяет два видео с затуханием
    Первое видео затухает в течение time секунд
    video1, video2 - обрабатываемые видео
    time - время перехода (эффекта)
    """
    clip1 = VideoFileClip(video1)
    clip2 = VideoFileClip(video2)

    # создание эффекта
    transition = CompositeVideoClip([clip1.crossfadeout(time)])

    final_clip = concatenate_videoclips([transition, clip2])
    final_clip.write_videofile("output_black_fade_out_concat.mp4")

In [123]:
make_black_fade_out_concat('test2.mp4', 'test1.mp4', 2)

Moviepy - Building video output_black_fade_out_concat.mp4.
MoviePy - Writing audio in output_black_fade_out_concatTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_black_fade_out_concat.mp4



Moviepy - Done !
Moviepy - video ready output_black_fade_out_concat.mp4


### Объединение с затуханием (до черного кадра). Второе видео постепенно появляется из черного кадра. Не хватило времени на реализацию, тут использованы готовые функции

In [119]:
def make_black_fade_in_concat(video1, video2, time):
    """
    Функция объединяет два видео с затуханием
    Второе видео появляется из черного кадра в течение time
    video1, video2 - обрабатываемые видео
    time - время перехода (эффекта)
    """
    clip1 = VideoFileClip(video1)
    clip2 = VideoFileClip(video2)

    # создание эффекта
    transition = CompositeVideoClip([clip2.crossfadein(time)])

    final_clip = concatenate_videoclips([clip1, transition])
    final_clip.write_videofile("output_black_fade_in_concat.mp4")

In [121]:
make_black_fade_in_concat('test2.mp4', 'test1.mp4', 4)

Moviepy - Building video output_black_fade_concat.mp4.
MoviePy - Writing audio in output_black_fade_concatTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_black_fade_concat.mp4



Moviepy - Done !
Moviepy - video ready output_black_fade_concat.mp4


In [120]:
cv2.destroyAllWindows()

### Объединение с размытием на 2 секунды. Результат - output_blur_concat.mp4

In [24]:
# blur transition between 2 videos
def make_blur_concat(video1, video2, coeff1, coeff2, blur):
    """
    Функция для объединения видео с эффектом блюра
    video1, video2 - два объединяемых видео
    coeff1, coeff2 - коэффициент блюра - больше коэфф = сильнее блюр
    blur - общий параметр для размытия 
    """
    def make_focused(clip, coeff):
        """
        Функция блюрит клип
        clip - видео
        coeff - коэффициент размытия для видео
        """
        # вычисляем коэффициент размытия
        blur_factor = coeff / blur
        # устанавливаем эффект размытия
        blurred = clip.fl_image(lambda img: gaussian_filter(img, sigma=blur_factor))
        # возвращаем видео с эффектом фокусировки
        return blurred

    # загружаем два видеофайла
    clip1 = VideoFileClip(video1)
    clip2 = VideoFileClip(video2)
    
    if clip1.duration <= 2 or clip2.duration <= 2:
        return 'Too short video'
    # создаем видео с эффектом размытия для первого видео
    focused1 = make_focused(clip1, coeff1)

    # создаем видео с эффектом размытия для второго видео
    focused2 = make_focused(clip2, coeff2)

    # обрезаем каждый клип до двух секунд
    focused1 = focused1.subclip(2,4)
    focused2 = focused2.subclip(0, 2)

    # объединяем два клипа
    final_clip = concatenate_videoclips([clip1.subclip(0, 2), focused1, focused2, clip2.subclip(2, clip2.duration)])

    # сохраняем итоговый клип в файл
    final_clip.write_videofile("output_blur_concat.mp4")


### Объединим два видео с эффектом перехода в виде размытия. Результат - output_blur_concat.mp4

In [25]:
make_blur_concat('test.mp4', 'test1.mp4', 4, 2, 2)

Moviepy - Building video output_blur_concat.mp4.
MoviePy - Writing audio in output_blur_concatTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_blur_concat.mp4



Moviepy - Done !
Moviepy - video ready output_blur_concat.mp4


### Дополнительный эффект для группы. Строго не судите... Используется готовый  конкат т.к. наш работал не очень хорошо в данной задаче

In [133]:
def make_parallel_concat(video1, video2, fps, shift=0.5):
    """
    Функция объединяет два видео для параллельного воспроизведения
    Без ресайза, со сдвигом
    После окончания видео с меньшей длиной на весь экарн воспроизводится
    остаток видео с большей длиной 
    video1, video2 - видео для объединения
    длительность video1 должна быть < длительности video2
    shift - пропорции видео при воспроизведении. Чем больше shift,
    тем сильнее первое видео сдвинуто вправо
    """

    clip1 = VideoFileClip(video1)
    clip2 = VideoFileClip(video2)

    # узнаем длительность видео
    duration = clip1.duration

    # создаем новый видео-клип
    new_clip = CompositeVideoClip([
        clip1.set_position((0, 0)),
        clip2.set_position((int(shift * clip2.w), 0))
    ], size=clip1.size).subclip(0, duration)

    final_clip = concatenate_videoclips([new_clip, clip2.subclip(new_clip.duration, clip2.duration)])
    # сохраняем результат
    final_clip.write_videofile('output_parallel_concat.mp4', fps)


In [137]:
make_parallel_concat('test1.mp4', 'test2.mp4', 60, 0.5)

Moviepy - Building video output_parallel_concat.mp4.
MoviePy - Writing audio in output_parallel_concatTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_parallel_concat.mp4



Moviepy - Done !
Moviepy - video ready output_parallel_concat.mp4


### Объединение двух видео с эффектом перехода в виде сдвига

In [138]:
def move_out_left(video):
    """
    Функция для реализации постепенного сдвига видео влево
    Реализуется выход видео за кадр
    video - обрабатываемое видео
    """
    cap = cv2.VideoCapture(video)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter('output_out.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 60, (width, height))

    # Начальное значение сдвига
    shift_start = 0

    # Максимальное значение сдвига
    shift_max = 2500

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    time_max = frame_count/fps

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Вычисление текущего времени
        time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000

        # Вычисление сдвига на основе времени
        shift = int(np.interp(time, [0, time_max], [shift_start, shift_max]))

        # Сдвиг кадра на shift пикселей влево
        shifted_frame = frame[:, shift:, :]
        black_frame = frame[:, :shift, :] * 0

        # Объединение сдвинутого и черного кадров
        final_frame = cv2.hconcat([shifted_frame, black_frame])

        # Запись кадра в выходной файл
        out.write(final_frame)

    # Освобождение ресурсов
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [139]:
def move_in_left(video):
    """
    Функция для реализациия постепеного сдвига видео влево
    Реализуется вход видео в кадр
    """
    cap = cv2.VideoCapture(video)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter('output_in.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 60, (width, height))

    # Начальное значение сдвига
    shift_start = 0

    # Максимальное значение сдвига
    shift_max = -2500

    # Время для достижения максимального сдвига
    time_max = 5

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Вычисление текущего времени
        time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000

        # Вычисление сдвига на основе времени
        shift = int(np.interp(time, [0, time_max], [shift_start, shift_max]))

        # Сдвиг кадра на shift пикселей влево
        shifted_frame = frame[:, shift:, :]
        black_frame = frame[:, :shift, :] * 0

        # Объединение сдвинутого и черного кадров
        final_frame = cv2.hconcat([black_frame, shifted_frame])

        # Запись кадра в выходной файл
        out.write(final_frame)

    # Освобождение ресурсов
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [140]:
def make_left_movement(video1, video2):
    """
    Функция, которая реализует объединение двух 
    видео со сдвигом первого и наплывом второго
    video1, video2 - обрабатываемые видео
    """
    move_out_left(video1)
    move_in_left(video2)
    make_concat('output_out.mp4', 'output_in.mp4')
    cv2.destroyAllWindows()

In [141]:
make_left_movement('test1.mp4', 'test.mp4')

### Объединение с затуханием (прозрачность). Результат - output_fade_concat.mp4

In [42]:
#смена изображений затуханием, только одинаковые размеры

def find_first_frame(video):
    """
    Функция сохраняет первый кадр видео
    """
    cap = cv2.VideoCapture(video)

    # переходим к первому кадру
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # читаем первый кадр
    ret, first_frame = cap.read()

    # сохраняем первый кадр как картинку
    cv2.imwrite('first_frame.jpg', first_frame)
    
    cap.release()
    cv2.destroyAllWindows()
    
def find_last_frame(video):
    """
    Функция сохраняет последний кадр видео
    """
    folder, _ = os.path.splitext(video)
    folder += '_cut'
    frame_cut(video)
    last_frame = Image.open(sorted(glob.glob(folder + '/*.jpg'), reverse=True)[0])
    last_frame.save('last_frame.jpg')

def get_fps(video):
    """
    Функция возвращает фпс видео
    """
    return int(VideoFileClip(video).fps)


def make_fade_concat(video1, video2):
    """
    Функция объединяет два видео с эффектом затухания
    """
    find_last_frame(video1)
    find_first_frame(video2)
    img1 = cv2.imread('first_frame.jpg')
    img2 = cv2.imread('last_frame.jpg')
    fps = min(get_fps(video1), get_fps(video2))
    # определение размеров кадра
    height, width, _ = img1.shape

    # количество кадров в итоговом видео (переход)
    num_frames = 100

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height), True)

    # цикл по количеству кадров
    for i in range(num_frames):
        # вычисление веса для каждой картинки
        alpha = i / num_frames
        beta = 1.0 - alpha

        # создание текущего кадра
        curr_frame = cv2.addWeighted(img1, alpha, img2, beta, 0.0)

        # запись текущего кадра в видео-файл
        video.write(curr_frame)

    video.release()
    clip1 = VideoFileClip(video1)
    clip2 = VideoFileClip('output.mp4')
    clip3 = VideoFileClip(video2)
    final_video = concatenate_videoclips([clip1, clip2, clip3])
    final_video.write_videofile("output_fade_concat.mp4", fps)
    cv2.destroyAllWindows()

In [43]:
make_fade_concat('test2.mp4', 'test1.mp4')

Moviepy - Building video output_fade_concat.mp4.
MoviePy - Writing audio in output_fade_concatTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_fade_concat.mp4



Moviepy - Done !
Moviepy - video ready output_fade_concat.mp4
